# Emirhan BULUT Voice Detection AI Software

In [2]:
!unzip data_emir.zip

Archive:  data_emir.zip
   creating: data_load_emir/
   creating: data_load_emir/other/
  inflating: data_load_emir/other/exercise_bike.wav  
  inflating: data_load_emir/other/pink_noise.wav  
   creating: data_load_emir/sentences/
  inflating: data_load_emir/sentences/Recording (6)-[AudioTrimmer.com] (1).wav  
  inflating: data_load_emir/sentences/Recording (6)-[AudioTrimmer.com].wav  
  inflating: data_load_emir/sentences/Recording (7)-[AudioTrimmer.com] (1).wav  
  inflating: data_load_emir/sentences/Recording (7)-[AudioTrimmer.com].wav  
   creating: data_load_emir/walking/
  inflating: data_load_emir/walking/1.wav  
  inflating: data_load_emir/walking/2.wav  
  inflating: data_load_emir/walking/3.wav  
  inflating: data_load_emir/walking/4.wav  
  inflating: data_load_emir/walking/5.wav  
  inflating: data_load_emir/walking/6.wav  
   creating: data_load_emir/_background_noise_/
  inflating: data_load_emir/_background_noise_/10convert.com_Audience-Claps_daSG5fwdA7o.wav  
  inflati

In [47]:
import os
import shutil
import numpy as np
import tensorflow as tf
from tensorflow import keras
from pathlib import Path
from IPython.display import display, Audio
DATASET_ROOT = os.path.join(os.path.expanduser("~"), "/content/data_load_emir")
AUDIO_SUBFOLDER = "datas"
NOISE_SUBFOLDER = "noises"
DATASET_AUDIO_PATH = os.path.join(DATASET_ROOT, AUDIO_SUBFOLDER)
DATASET_NOISE_PATH = os.path.join(DATASET_ROOT, NOISE_SUBFOLDER)
VALID_SPLIT = 0.1
SHUFFLE_SEED = 36
SAMPLING_RATE = 16000
SCALE = 0.3
BATCH_SIZE = 128
EPOCHS = 20

In [33]:
if os.path.exists(DATASET_AUDIO_PATH) is False:
    os.makedirs(DATASET_AUDIO_PATH)
if os.path.exists(DATASET_NOISE_PATH) is False:
    os.makedirs(DATASET_NOISE_PATH)
for folder in os.listdir(DATASET_ROOT):
    if os.path.isdir(os.path.join(DATASET_ROOT, folder)):
        if folder in [AUDIO_SUBFOLDER, NOISE_SUBFOLDER]:
            continue
        elif folder in ["other", "_background_noise_"]:
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_NOISE_PATH, folder),
            )
        else:
            shutil.move(
                os.path.join(DATASET_ROOT, folder),
                os.path.join(DATASET_AUDIO_PATH, folder),
            )

In [34]:
noise_paths = []
for subdir in os.listdir(DATASET_NOISE_PATH):
    subdir_path = Path(DATASET_NOISE_PATH) / subdir
    if os.path.isdir(subdir_path):
        noise_paths += [
            os.path.join(subdir_path, filepath)
            for filepath in os.listdir(subdir_path)
            if filepath.endswith(".wav")
        ]

print(
    "{} files found in {} directories".format(
        len(noise_paths), len(os.listdir(DATASET_NOISE_PATH))
    )
)

6 files found in 2 directories


Resample all noise samples to 16000 Hz

In [35]:
command = (
    "for dir in `ls -1 " + DATASET_NOISE_PATH + "`; do "
    "for file in `ls -1 " + DATASET_NOISE_PATH + "/$dir/*.wav`; do "
    "sample_rate=`ffprobe -hide_banner -loglevel panic -show_streams "
    "$file | grep sample_rate | cut -f2 -d=`; "
    "if [ $sample_rate -ne 16000 ]; then "
    "ffmpeg -hide_banner -loglevel panic -y "
    "-i $file -ar 16000 temp.wav; "
    "mv temp.wav $file; "
    "fi; done; done"
)
os.system(command)

def load_noise_sample(path):
    sample, sampling_rate = tf.audio.decode_wav(
        tf.io.read_file(path), desired_channels=1
    )
    if sampling_rate == SAMPLING_RATE:
        slices = int(sample.shape[0] / SAMPLING_RATE)
        sample = tf.split(sample[: slices * SAMPLING_RATE], slices)
        return sample
    else:
        print("Sampling rate for {} is incorrect. Ignoring it".format(path))
        return None


noises = []
for path in noise_paths:
    sample = load_noise_sample(path)
    if sample:
        noises.extend(sample)
noises = tf.stack(noises)

print(
    "{} noise files were split into {} noise samples where each is {} sec. long".format(
        len(noise_paths), noises.shape[0], noises.shape[1] // SAMPLING_RATE
    )
)

6 noise files were split into 354 noise samples where each is 1 sec. long


In [36]:
def paths_and_labels_to_dataset(audio_paths, labels):
    path_ds = tf.data.Dataset.from_tensor_slices(audio_paths)
    audio_ds = path_ds.map(lambda x: path_to_audio(x))
    label_ds = tf.data.Dataset.from_tensor_slices(labels)
    return tf.data.Dataset.zip((audio_ds, label_ds))


def path_to_audio(path):
    audio = tf.io.read_file(path)
    audio, _ = tf.audio.decode_wav(audio, 1, SAMPLING_RATE)
    return audio


def add_noise(audio, noises=None, scale=0.5):
    if noises is not None:
        tf_rnd = tf.random.uniform(
            (tf.shape(audio)[0],), 0, noises.shape[0], dtype=tf.int32
        )
        noise = tf.gather(noises, tf_rnd, axis=0)
        prop = tf.math.reduce_max(audio, axis=1) / tf.math.reduce_max(noise, axis=1)
        prop = tf.repeat(tf.expand_dims(prop, axis=1), tf.shape(audio)[1], axis=1)
        audio = audio + noise * prop * scale

    return audio


def audio_to_fft(audio):
    audio = tf.squeeze(audio, axis=-1)
    fft = tf.signal.fft(
        tf.cast(tf.complex(real=audio, imag=tf.zeros_like(audio)), tf.complex64)
    )
    fft = tf.expand_dims(fft, axis=-1)
    return tf.math.abs(fft[:, : (audio.shape[1] // 2), :])
class_names = os.listdir(DATASET_AUDIO_PATH)
print("Our class names: {}".format(class_names,))

audio_paths = []
labels = []
for label, name in enumerate(class_names):
    print("Processing voices {}".format(name,))
    dir_path = Path(DATASET_AUDIO_PATH) / name
    voice_sample_paths = [
        os.path.join(dir_path, filepath)
        for filepath in os.listdir(dir_path)
        if filepath.endswith(".wav")
    ]
    audio_paths += voice_sample_paths
    labels += [label] * len(voice_sample_paths)

print(
    "Found {} files belonging to {} classes.".format(len(audio_paths), len(class_names))
)

rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(audio_paths)
rng = np.random.RandomState(SHUFFLE_SEED)
rng.shuffle(labels)

num_val_samples = int(VALID_SPLIT * len(audio_paths))
print("Using {} files for training.".format(len(audio_paths) - num_val_samples))
train_audio_paths = audio_paths[:-num_val_samples]
train_labels = labels[:-num_val_samples]

print("Using {} files for validation.".format(num_val_samples))
valid_audio_paths = audio_paths[-num_val_samples:]
valid_labels = labels[-num_val_samples:]

train_ds = paths_and_labels_to_dataset(train_audio_paths, train_labels)
train_ds = train_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

valid_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
valid_ds = valid_ds.shuffle(buffer_size=32 * 8, seed=SHUFFLE_SEED).batch(32)

train_ds = train_ds.map(
    lambda x, y: (add_noise(x, noises, scale=SCALE), y),
    num_parallel_calls=tf.data.AUTOTUNE,
)

train_ds = train_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)

valid_ds = valid_ds.map(
    lambda x, y: (audio_to_fft(x), y), num_parallel_calls=tf.data.AUTOTUNE
)
valid_ds = valid_ds.prefetch(tf.data.AUTOTUNE)

Our class names: ['sentences', 'walking']
Processing voices sentences
Processing voices walking
Found 10 files belonging to 2 classes.
Using 9 files for training.
Using 1 files for validation.


In [45]:
def build_model(input_shape, num_classes):
    inputs = keras.layers.Input(shape=input_shape, name="input_name")

    x = keras.layers.Conv1D(16, 2)(inputs)
    x = keras.layers.MaxPool1D(pool_size=2, strides=2)(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(32, activation="relu")(x)
    x = keras.layers.Dense(64, activation="relu")(x)

    outputs = keras.layers.Dense(num_classes, activation="sigmoid", name="output")(x)

    return keras.models.Model(inputs=inputs, outputs=outputs)


model = build_model((SAMPLING_RATE // 2, 1), len(class_names))

model.summary()

model.compile(
    optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

model_save_filename = "model_emir.h5"

earlystopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
mdlcheckpoint_cb = keras.callbacks.ModelCheckpoint(
    model_save_filename, monitor="val_accuracy", save_best_only=True
)

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_name (InputLayer)     [(None, 8000, 1)]         0         
                                                                 
 conv1d_9 (Conv1D)           (None, 7999, 16)          48        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 3999, 16)         0         
 )                                                               
                                                                 
 flatten_7 (Flatten)         (None, 63984)             0         
                                                                 
 dense_14 (Dense)            (None, 32)                2047520   
                                                                 
 dense_15 (Dense)            (None, 64)                2112      
                                                           

In [48]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=valid_ds,
    callbacks=[earlystopping_cb, mdlcheckpoint_cb],
)

Epoch 1/20
1/1 [==============================] - 1s 747ms/step - loss: 0.2289 - accuracy: 1.0000 - val_loss: 0.4588 - val_accuracy: 1.0000
Epoch 2/20
1/1 [==============================] - 0s 338ms/step - loss: 0.2162 - accuracy: 1.0000 - val_loss: 0.4218 - val_accuracy: 1.0000
Epoch 3/20
1/1 [==============================] - 0s 291ms/step - loss: 0.2221 - accuracy: 1.0000 - val_loss: 0.4171 - val_accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 292ms/step - loss: 0.1952 - accuracy: 1.0000 - val_loss: 0.2951 - val_accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 292ms/step - loss: 0.1588 - accuracy: 1.0000 - val_loss: 0.2168 - val_accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 300ms/step - loss: 0.1824 - accuracy: 1.0000 - val_loss: 0.1837 - val_accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 286ms/step - loss: 0.1726 - accuracy: 1.0000 - val_loss: 0.1916 - val_accuracy: 1.0000
Epoch 8/20
1/1 [====

In [49]:
print(model.evaluate(valid_ds))

1/1 [==============================] - 0s 77ms/step - loss: 0.0842 - accuracy: 1.0000
[0.0842311903834343, 1.0]


In [50]:
display_example = 1

test_ds = paths_and_labels_to_dataset(valid_audio_paths, valid_labels)
test_ds = test_ds.shuffle(buffer_size=BATCH_SIZE * 8, seed=SHUFFLE_SEED).batch(
    BATCH_SIZE
)

test_ds = test_ds.map(lambda x, y: (add_noise(x, noises, scale=SCALE), y))

for audios, labels in test_ds.take(1):
    ffts = audio_to_fft(audios)
    y_pred = model.predict(ffts)
    rnd = np.random.randint(0, 1, display_example)
    audios = audios.numpy()[rnd, :, :]
    labels = labels.numpy()[rnd]
    y_pred = np.argmax(y_pred, axis=-1)[rnd]

    for index in range(display_example):
        print(
            "Voice Type:\33{} {}\33[0m\tPredicted:\33{} {}\33[0m".format(
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[labels[index]],
                "[92m" if labels[index] == y_pred[index] else "[91m",
                class_names[y_pred[index]],
            )
        )
        display(Audio(audios[index, :, :].squeeze(), rate=SAMPLING_RATE))

Voice Type: walking	Predicted: walking


In [51]:
!zip /content/model_emir.zip /content/model_emir.h5

updating: content/model_emir.h5 (deflated 13%)
